In [70]:
import torch
import numpy as np

In [102]:
def epe_v1(flow_pred, flow_gt):
    epe_img = torch.sum((flow_pred - flow_gt) ** 2, dim=1).sqrt()
    return epe_img.mean()

def epe_v2(flow_pred, flow_gt, valid_mask):
    epe_img = torch.sum((flow_pred - flow_gt) ** 2, dim=1).sqrt()
    epe_img = epe_img.view(-1)[valid_mask.view(-1)]
    return epe_img.mean()

def epe_v3(flow_pred, flow_gt, valid_mask):
    epe_img = torch.sum((flow_pred - flow_gt) ** 2, dim=1).sqrt()
    return (valid_mask * epe_img).mean()

def epe_v3_fixed(flow_pred, flow_gt, valid_mask):
    epe_img = torch.sum((flow_pred - flow_gt) ** 2, dim=1).sqrt()
    return torch.sum(epe_img * valid_mask) / torch.sum(valid_mask)


def l1_loss(flow_pred, flow_gt, valid_mask=None):
    loss_img = torch.sum((flow_pred - flow_gt).abs(), dim=1)
    if valid_mask is not None:
        return (valid_mask * loss_img).mean()
    else:
        return loss_img.mean()

def l1_loss_fixed(flow_pred, flow_gt, valid_mask=None):
    loss_img = torch.sum((flow_pred - flow_gt).abs(), dim=1)
    if valid_mask is not None:
        return torch.sum(loss_img * valid_mask) / torch.sum(valid_mask)
    else:
        return loss_img.mean()

In [72]:
flow_gt = torch.ones((1, 2, 3, 3))
flow_gt

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [73]:
flow_pred = torch.ones((1, 2, 3, 3))
flow_pred[:, 0, 0, :] += 1
flow_pred[:, 0, 1, :] += 2
flow_pred[:, 0, 2, :] += 3
flow_pred

tensor([[[[2., 2., 2.],
          [3., 3., 3.],
          [4., 4., 4.]],

         [[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [81]:
valid_mask = torch.from_numpy(np.array([
    [0, 0, 0], 
    [1, 1, 1], 
    [1, 1, 1]]).astype(bool)
)
valid_mask

tensor([[False, False, False],
        [ True,  True,  True],
        [ True,  True,  True]])

In [75]:
epe_v1(flow_pred, flow_gt)

tensor(2.)

In [85]:
epe_v2(flow_pred, flow_gt, valid_mask)

tensor(2.5000)

In [77]:
epe_img = torch.sum((flow_pred - flow_gt) ** 2, dim=1).sqrt()
epe_img

tensor([[[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]]])

In [78]:
epe_vec = epe_img.view(-1)
epe_vec

tensor([1., 1., 1., 2., 2., 2., 3., 3., 3.])

In [82]:
valid_mask_vec = valid_mask.view(-1)
valid_mask_vec

tensor([False, False, False,  True,  True,  True,  True,  True,  True])

In [84]:
epe_vec[valid_mask_vec]

tensor([2., 2., 2., 3., 3., 3.])

In [87]:
epe_v3(flow_pred, flow_gt, valid_mask)

tensor(1.6667)

In [88]:
epe_img * valid_mask

tensor([[[0., 0., 0.],
         [2., 2., 2.],
         [3., 3., 3.]]])

In [90]:
epe_v3_fixed(flow_pred, flow_gt, valid_mask)

tensor(2.5000)

In [103]:
l1_loss(flow_pred, flow_gt, valid_mask)

tensor(1.6667)

In [104]:
l1_loss_fixed(flow_pred, flow_gt, valid_mask)

tensor(2.5000)

In [106]:
loss_img = torch.sum((flow_pred - flow_gt).abs(), dim=1)
loss_img

tensor([[[1., 1., 1.],
         [2., 2., 2.],
         [3., 3., 3.]]])

In [107]:
loss_img.mean()

tensor(2.)

In [108]:
valid_mask * loss_img

tensor([[[0., 0., 0.],
         [2., 2., 2.],
         [3., 3., 3.]]])

In [109]:
torch.sum(valid_mask * loss_img)

tensor(15.)

In [110]:
torch.sum(valid_mask)

tensor(6)